In [1]:
import json
import os
from collections import defaultdict

In [2]:
path = "/scratch/zar8jw/Audio/segments_v1"
res = defaultdict(dict)
for root, dirs, files in os.walk(path):
    name = os.path.basename(root)
    for file in files:
        if file == "transcript.json":
            text = ""
            with open(os.path.join(root, file), 'r') as f:
                cur_transcripts = json.load(f)
            for i in range(len(cur_transcripts)):
                text += cur_transcripts[i]["transcript"]
            res[name]["pred"] = text

In [3]:
res

defaultdict(dict,
            {'GX010322_encoded_trimmed': {'pred': 'Checking pulse. No pulse, no breathing. Starting CPR.Analyzing heart rhythm. Do not touch the patient.Do not touch the patient. Analyzing heart rhythm.Shock advised.Do not touch the patientPress the flashing shock button.Shock delivered'},
             'GX010324_encoded_trimmed': {'pred': 'No pulse, no breathing. Starting CPR.Press pads firmly on skinDo not touch the patient. Analyzing heart rhythm.Do not touch the patient. Analyzing heart rhythm.Shock advised.Do not touch the patientPress the flashing shock button.Shock'},
             'GX010323_encoded_trimmed': {'pred': 'No pulse. Starting CPR.Adult patient.  Pads on.  Charging...clear!  Analyzing...Press pads firmly on chest.Do not touch the patient.Analyzing heart rhythm...Analyzing heart rhythm.Shock advised.Do not touch the patientPress the flashing shock button.Shock delivered.'},
             'GX010364_encoded_trimmed': {'pred': 'No breath.  [Sound of chest c

In [4]:
file_names = [each.replace("_trimmed", "_human.json") for each in list(res.keys())]
path = "/scratch/zar8jw/Audio/manual_check_transcripts"
for root, dirs, files in os.walk(path):
    for file in files:
        if file in file_names:
            with open(os.path.join(root, file), 'r') as f:
                gt_file = json.load(f)
            
            gt_text = ""
            for i in range(len(gt_file)):
                gt_text += gt_file[i]["Utterance"] + " "
            
            res[file.replace("_human.json", "_trimmed")]["gt"] = gt_text

In [5]:
res

defaultdict(dict,
            {'GX010322_encoded_trimmed': {'pred': 'Checking pulse. No pulse, no breathing. Starting CPR.Analyzing heart rhythm. Do not touch the patient.Do not touch the patient. Analyzing heart rhythm.Shock advised.Do not touch the patientPress the flashing shock button.Shock delivered',
              'gt': 'In progress Checking pulse No pulse, no breathing Starting CPR Give me two breaths Starting 10 20 30, give two breaths 10 20 30, give two breaths Stop CPR. Do not touch the patient, analyzing heart rhythm. Do not touch the patient analyzing heart rhythm. Shock advised. Do not touch the patient. Everyone clear. Press the flashing shock button. Shock delivered. Begin CPR. 10 20 30, give two breaths '},
             'GX010324_encoded_trimmed': {'pred': 'No pulse, no breathing. Starting CPR.Press pads firmly on skinDo not touch the patient. Analyzing heart rhythm.Do not touch the patient. Analyzing heart rhythm.Shock advised.Do not touch the patientPress the flashing

In [6]:
import jiwer
from collections import defaultdict
import inflect
import string
# Create an inflect engine
p = inflect.engine()

# Define a custom function to replace digits with words
def digits_to_words(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    words = text.split()
    converted_words = [p.number_to_words(word) if word.isdigit() else word for word in words]
    return " ".join(converted_words)

def get_wer(gt, pred):

    gt_transformed = digits_to_words(gt)
    pred_transformed = digits_to_words(pred)

    transforms = jiwer.Compose(
        [
            jiwer.ExpandCommonEnglishContractions(),
            jiwer.RemoveEmptyStrings(),
            jiwer.ToLowerCase(),
            jiwer.RemoveMultipleSpaces(),
            jiwer.Strip(),
            jiwer.RemovePunctuation(),
            jiwer.ReduceToListOfListOfWords(),
        ]
    )
    wer = jiwer.wer(
                gt_transformed,
                pred_transformed,
                truth_transform=transforms,
                hypothesis_transform=transforms,
            )
    return wer

In [7]:
import numpy as np
all_wer = []
for k, metadata in res.items():
    wer = get_wer(metadata["gt"], metadata["pred"])
    all_wer.append(wer)
    print(k, wer)
print(f"avg: {np.average(all_wer)}")

GX010322_encoded_trimmed 0.6521739130434783
GX010324_encoded_trimmed 0.6533333333333333
GX010323_encoded_trimmed 0.7428571428571429
GX010364_encoded_trimmed 0.8737864077669902
GX010321_encoded_trimmed 0.8243243243243243
GX010332_encoded_trimmed 0.88
GX010318_encoded_trimmed 0.5
GX010336_encoded_trimmed 0.7446808510638298
GX010319_encoded_trimmed 0.43157894736842106
avg: 0.7003038799730578
